### 2.1.1 启动线程

In [1]:
#include <iostream>
#include <thread>

In [2]:
namespace n2 {
    void do_something()
    {
        std::cout << "do_something" << "\n";
    }

    void do_something_else()
    {
        std::cout << "do_something_else" << "\n";
    }

    class background_task
    {
    public:
        void operator()() const
        {
            do_something();
            do_something_else();
        }
    };
}

In [3]:
{
    using namespace n2;
    background_task f;
    std::thread my_thread(f);

    my_thread.join();
}

do_something
do_something_else


有件事需要注意，当把函数对象传入到线程构造函数中时，需要避免“最令人头痛的语法解析”(C++’s most vexing parse, 中文简介)。如果你传递了一个临时变量，而不是一个命名的变量；C++编译器会将其解析为函数声明，而不是类型对象的定义。

In [4]:
/*
这里相当与声明了一个名为my_thread的函数，这个函数带有一个参数(函数指针指向没有参数并返回background_task对象的函数)，
返回一个 std::thread 对象的函数，而非启动了一个线程。
*/
{
    using namespace n2;
    std::thread my_thread(background_task());
}

input_line_6:8:26: warning: parentheses were disambiguated as a function declaration [-Wvexing-parse]
    std::thread my_thread(background_task());
                         ^~~~~~~~~~~~~~~~~~~
input_line_6:8:27: note: add a pair of parentheses to declare a variable
    std::thread my_thread(background_task());
                          ^
                          (                )


In [5]:
// 使用在前面命名函数对象的方式，或使用多组括号①，或使用新统一的初始化语法②，可以避免这个问题。
{
    using namespace n2;
    
    background_task f;
    std::thread my_thread0(f);

    std::thread my_thread1((background_task())); // 1
    std::thread my_thread2{background_task()}; // 2
    
    my_thread0.join();
    my_thread1.join();
    my_thread2.join();
}

do_something
do_something_else
do_something
do_something_else
do_something
do_something_else


In [6]:
// 使用lambda表达式也能避免这个问题
{
    using namespace n2;
    std::thread my_thread([]{
        do_something();
        do_something_else();
    });
    my_thread.join();
}

do_something
do_something_else
